<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess_sku.py -O preprocess_sku.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/model_utils.py -O model_utils.py
! unzip -o ml-1m.zip 

--2022-10-17 15:05:35--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  24.3MB/s    in 0.2s    

2022-10-17 15:05:36 (24.3 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-10-17 15:05:36--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess_sku.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6756 (6.6K) [text/plain]
Saving to: ‘preprocess_sku.py’

preprocess_sku.py   100%[===================>]

In [ ]:
# ! pip install -q deepmatch

     |████████████████████████████████| 147 kB 46.3 MB/s 
     |████████████████████████████████| 2.9 MB 61.2 MB/s 


In [43]:
import pandas as pd
from ast import literal_eval
from preprocess_sku import gen_data_set, gen_model_input, gen_data_set_timesplit
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from model_utils import DNN, EmbeddingIndex, NoMask, PoolingLayer, l2_normalize,\
 reduce_mean, SampledSoftmaxLayer, get_item_embedding, sampledsoftmaxloss

In [5]:
%load_ext autoreload
%autoreload 2

# Data

In [ ]:
# data_path = "./"

# unames = ['user_id','gender','age','occupation','zip']
# user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
# rnames = ['user_id','sku_number','rating','timestamp']
# ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
# mnames = ['sku_number','title','genres']
# movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
# movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

# data = pd.merge(pd.merge(ratings,movies),user).iloc[:10000] # taking 1w
# print(data.shape)
# print(data.user_id.nunique())
# print(data.sku_number.nunique())
# data.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


(10000, 10)
46
2175


,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation,F,1,10,48067


### Sales data

In [16]:
data = pd.read_csv('sample_data/mik_sales_sp_10052022.csv', index_col = False, 
                   parse_dates = ['trans_date','created_time']
                   )
# data['most_recent_purchased_sku'] = data['most_recent_purchased_sku'].apply(literal_eval)
# data['most_recent_view_sku'] = data['most_recent_view_sku'].apply(literal_eval)
# data['hist_sku_number'] = data['most_recent_purchased_sku'] + data['most_recent_view_sku']
data.shape

(23206, 14)

In [17]:
data['user_id'].nunique(), data['sku_number'].nunique()

(200, 8036)

In [33]:
data['trans_date'].max(),data['trans_date'].min(), data['trans_date'].max() - data['trans_date'].min()

(Timestamp('2022-10-02 00:00:00'),
 Timestamp('2022-04-02 00:00:00'),
 Timedelta('183 days 00:00:00'))

In [58]:
import datetime
# test_start_date = pd.Timestamp(datetime.datetime.today().date() - datetime.timedelta(6))
test_start_date = data['trans_date'].max() - datetime.timedelta(6)

test_start_date

Timestamp('2022-09-26 00:00:00')

In [18]:
data.columns

Index(['data_source', 'user_id', 'sku_number', 'qty', 'trans_date',
       'created_time', 'email_address', 'geo_zip',
       'most_frequent_device_class_general', 'total_order_value',
       'total_coupon_value', 'most_recent_purchased_sku',
       'most_recent_view_sku', 'category_path'],
      dtype='object')

In [9]:
data = data[['data_source', 'user_id', 'sku_number', 'qty', 'trans_date',
       'created_time', 'email_address', 'geo_zip','most_frequent_device_class_general','category_path']]

In [19]:
data['sku_number_org'] = data['sku_number']

### Item category mapping dat

In [10]:
df_item = pd.read_csv("sample_data/mik_item_07292022.csv", index_col=False)
df_item.shape

FileNotFoundError: ignored

In [11]:
df_item.head(2)

NameError: ignored

In [ ]:
data['most_recent_view_sku'].apply(lambda x: len(x)).describe()

# Feature Engineering

In [35]:
sparse_features = ['sku_number', 'category_path',
                   'user_id', 'geo_zip','most_frequent_device_class_general']

SEQ_LEN = 50
NUMBER_NEG_SAMPLE = 1 # used for generate 1:1 positive:negative samples


In [36]:
# view_item_set = list(set(data['most_recent_view_sku'].sum()))
# full_item_set = list(set(list(data['sku_number'].unique()) + view_item_set))
# len(full_item_set)

In [37]:
feature_max_idx = {}
for feature in sparse_features:
    # if feature == 'sku_number':      
    #   lbe_sku = LabelEncoder()
    #   lbe_sku.fit(full_item_set)
    #   data[feature] = lbe_sku.transform(data[feature]) + 1
    #   feature_max_idx[feature] = data[feature].max() + 1
    # else:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1 # add one to all the encoded categories labels
    feature_max_idx[feature] = data[feature].max() + 1

In [38]:
feature_max_idx

{'sku_number': 8037,
 'category_path': 795,
 'user_id': 201,
 'geo_zip': 186,
 'most_frequent_device_class_general': 3}

In [39]:
data.head(2)

,data_source,user_id,sku_number,qty,trans_date,created_time,email_address,geo_zip,most_frequent_device_class_general,total_order_value,total_coupon_value,most_recent_purchased_sku,most_recent_view_sku,category_path,sku_number_org
8530,ADOBE,99,3075,1,2022-04-02,2022-04-02 03:13:51,charrison711@aol.com,171,2,1082.94,0.0,"['10686776', '10685948', '10685944', '10666327...","['10548355', '10401939', '10367556', '10547844...",191,10472573
8547,ADOBE,99,2387,2,2022-04-02,2022-04-02 03:13:51,charrison711@aol.com,171,2,1082.94,0.0,"['10686776', '10685948', '10685944', '10666327...","['10548355', '10401939', '10367556', '10547844...",188,10402085


In [40]:
user_profile = data[['user_id', 'geo_zip', 'most_frequent_device_class_general']].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["sku_number"]].drop_duplicates('sku_number')
user_profile.shape, item_profile.shape

((200, 2), (8036, 1))

In [41]:
# user_item_list = data.groupby("user_id")['sku_number'].apply(list)
user_item_list = data.groupby("user_id")['sku_number'].apply(list)

user_item_list.head()

user_id
1    [6090, 5555, 6082, 6091, 5559, 6084, 6086, 554...
2    [3709, 3720, 1912, 3722, 1888, 3511, 3708, 370...
3    [2294, 940, 938, 1642, 736, 735, 6729, 3726, 6...
4    [3721, 3728, 3719, 3725, 4677, 3511, 3728, 372...
5    [1439, 215, 4271, 241, 213, 4270, 210, 992, 19...
Name: sku_number, dtype: object

In [52]:
data_sp = data[data['user_id'] == 99]
data_sp.shape

(186, 15)

In [53]:
data_sp.sort_values("created_time", inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [59]:
train_set, test_set = gen_data_set_timesplit(data, test_start_date, SEQ_LEN, NUMBER_NEG_SAMPLE)

100%|██████████| 200/200 [00:00<00:00, 215.47it/s]

7 7


In [ ]:
"# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - label y
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating
# train_set[12]

In [90]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess_sku.py:97: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_genres = np.array([line[6] for line in train_set])


((44840,), (44840,), (586,), (586,))

In [92]:
train_model_input['user_id'].shape

(44840,)

# Create Embeddings

In [110]:
from collections import Counter
train_counter = Counter(train_model_input['sku_number'])
item_count = [train_counter.get(i,0) for i in range(feature_max_idx['sku_number'])]
len(train_counter), type(train_counter), len(item_count)

(8030, collections.Counter, 8037)

In [111]:
import tensorflow as tf
from tensorflow.python.keras.initializers import RandomNormal, Zeros
from tensorflow.python.keras.layers import Embedding, Input, Lambda
from tensorflow.python.keras.regularizers import l2
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

### Create Feature Embeddings



In [112]:
# item features
item_feature_columns = ['sku_number']
# User features
user_feature_columns = ['user_id', 'geo_zip', 'most_frequent_device_class_general',                        
                        'hist_sku_number',
                        'hist_category_path',
                        'hist_len'
                        ]


In [113]:
feature_max_idx

{'sku_number': 8037,
 'category_path': 795,
 'user_id': 201,
 'geo_zip': 186,
 'most_frequent_device_class_general': 3}

In [114]:
user_sparse_feature_columns = user_feature_columns[0:3]
user_sparse_feature_columns

['user_id', 'geo_zip', 'most_frequent_device_class_general']

In [115]:
user_seq_sparse_feature_columns = user_feature_columns[3:-1]
user_seq_sparse_feature_columns 

['hist_sku_number', 'hist_category_path']

In [116]:
# embedding input and output specification
embeddings_initializer = RandomNormal(mean=0.0, stddev=0.0001, seed=2020)
embedding_dim = 32
l2_reg = 1e-6
# use a dict to collect embeddings
sparse_embedding = {}

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [117]:
# user embeddings
for name in user_sparse_feature_columns: 
  emb = Embedding(
      input_dim = feature_max_idx[name], 
      output_dim = 16,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + name
  )
  emb.trainable = True
  sparse_embedding[name] = emb
for name in user_seq_sparse_feature_columns: 
   emb = Embedding(feature_max_idx[name[5:]], embedding_dim,
                            embeddings_initializer=embeddings_initializer,
                            embeddings_regularizer=l2(
                                l2_reg),
                            name='sparse_' + 'seq_emb_'  + name,
                            mask_zero=False) # True
   emb.trainable = True
   sparse_embedding[name] = emb

In [118]:
# item embeddings
item_feature_name = item_feature_columns[0] # only include item id 
emb = Embedding(
      input_dim = feature_max_idx[item_feature_name], 
      output_dim = embedding_dim,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + item_feature_name
  )
sparse_embedding[item_feature_name] = emb

In [119]:
sparse_embedding # include both user and item embedding layers

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f0dfdaad990>,
 'geo_zip': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f0dfd97c250>,
 'most_frequent_device_class_general': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f0e0099aad0>,
 'hist_sku_number': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f0dfd97cd10>,
 'hist_category_path': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f0dfe260090>,
 'sku_number': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f0dfd97c910>}

# Create User Inputs


## Create Inputs Tensor

In [120]:
# user features inputs
from collections import OrderedDict
user_features = OrderedDict()
for name in user_sparse_feature_columns:
  user_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
for name in user_seq_sparse_feature_columns:
  user_features[name] = Input(
                shape=(SEQ_LEN,), 
                name=name, 
                dtype="int32"
                )
length_name = 'hist_len'
if length_name in user_feature_columns:
  user_features[length_name] = Input((1,), name=length_name, dtype='int32')

In [121]:
user_inputs_list = list(user_features.values())
user_inputs_list

[<tf.Tensor 'user_id:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'geo_zip:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'most_frequent_device_class_general:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'hist_sku_number:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_category_path:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_len:0' shape=(None, 1) dtype=int32>]

In [122]:
# item feature inputs
item_features = OrderedDict()
for name in item_feature_columns:
  item_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
item_inputs_list = list(item_features.values())


In [123]:
item_inputs_list

[<tf.Tensor 'sku_number:0' shape=(None, 1) dtype=int32>]

## Embed Inputs

In [124]:
# Embed sparse inputs
from collections import defaultdict
user_sparse_embeded_input = defaultdict(list)

for name in user_sparse_feature_columns:
  user_sparse_embeded_input[name] = sparse_embedding[name](user_features[name])

In [125]:
# Embed the sequence sparse inputs, need to add pooling layer to it
seq_embeded_dict = defaultdict(list)

for name in user_seq_sparse_feature_columns:
  seq_embeded_dict[name] = sparse_embedding[name](user_features[name])

# from deepctr.layers.sequence import SequencePoolingLayer
# for name in user_seq_sparse_feature_columns:
#   user_seq_sparse_embeded_input[name] = SequencePoolingLayer('mean', supports_masking=False)([seq_embeded_dict[name], user_features[length_name]])

from tensorflow.keras.layers import AveragePooling1D
user_seq_sparse_embeded_input = defaultdict(list)
for name in user_seq_sparse_feature_columns:
  user_seq_sparse_embeded_input[name] = AveragePooling1D(pool_size=50, padding='valid')(seq_embeded_dict[name])

In [126]:
# vec = AveragePooling1D(pool_size=50, strides = 50, padding='valid')
# vec(seq_embeded_dict[name])

In [127]:
# from deepctr.layers.sequence import SequencePoolingLayer
# vec2 = SequencePoolingLayer('mean', supports_masking=False)
# vec2([seq_embeded_dict[name], 35])

In [128]:
# combine embeded inputs into a list
user_sparse_embedding_list = list(user_sparse_embeded_input.values()) + list(user_seq_sparse_embeded_input.values())
user_sparse_embedding_list

[<tf.Tensor 'sparse_emb_user_id/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_geo_zip/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_most_frequent_device_class_general/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'average_pooling1d/Squeeze:0' shape=(None, 1, 32) dtype=float32>,
 <tf.Tensor 'average_pooling1d_1/Squeeze:0' shape=(None, 1, 32) dtype=float32>]

## Combine Embeded Input (User only)

In [129]:
from tensorflow.python.keras.layers import Flatten, Concatenate, Layer, Add

In [130]:
user_dnn_input = Flatten()(Concatenate(axis=2)(user_sparse_embedding_list))
user_dnn_input

<tf.Tensor 'flatten/Reshape:0' shape=(None, 112) dtype=float32>

# Sepecify Model Layers

### User Layers

In [131]:
from tensorflow.keras import layers

In [132]:
user_dnn_hidden_units = (128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
# sampler_config=sampler_config
seed=1024

In [133]:
# from deepctr.layers import DNN
from model_utils import DNN

In [134]:
dnn_layers = DNN(hidden_units = user_dnn_hidden_units, 
                  activation='relu', 
                  l2_reg=0, 
                  dropout_rate=0, 
                  use_bn=False, 
                  output_activation='linear',
                  seed=1024
                  )



In [135]:
user_dnn_output = dnn_layers(user_dnn_input)
user_dnn_output = l2_normalize(user_dnn_output)
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

### Item Layer

In [136]:
# Create item index tensor
item_vocabulary_size = feature_max_idx[item_feature_name]
item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(item_features[item_feature_name])

# embed the item index input
item_embeding = sparse_embedding[item_feature_name]
item_embedding_weight = NoMask()(item_embeding(item_index))
pooling_item_embedding_weight = PoolingLayer()([item_embedding_weight])
pooling_item_embedding_weight = l2_normalize(pooling_item_embedding_weight)
pooling_item_embedding_weight


<tf.Tensor 'lambda_1/l2_normalize:0' shape=(8037, 32) dtype=float32>

In [137]:
pooling_item_embedding_weight.eval(session=K.get_session()).shape

(8037, 32)

In [138]:
# .eval(session=K.get_session())

### Output Layer

In [139]:
output = SampledSoftmaxLayer()(
    [pooling_item_embedding_weight, user_dnn_output, item_features[item_feature_name]])
output

<tf.Tensor 'sampled_softmax_layer/ExpandDims:0' shape=(None, 1) dtype=float32>

In [140]:
# from model_utils import SampledSoftmaxLayer
# sf = SampledSoftmaxLayer()

In [141]:
# from tensorflow.keras.layers import Softmax
# sf2 = Softmax()
# import numpy as np
# x = np.asarray([[1., 2., 1.]])
# x
# sf2(x).eval(session=K.get_session())

In [142]:
pooling_item_embedding_weight

<tf.Tensor 'lambda_1/l2_normalize:0' shape=(8037, 32) dtype=float32>

In [143]:
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

In [144]:
item_features[item_feature_name]

<tf.Tensor 'sku_number:0' shape=(None, 1) dtype=int32>

## Build Model

In [145]:
from tensorflow.python.keras.models import Model

In [146]:
model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

model.__setattr__("user_input", user_inputs_list)
model.__setattr__("user_embedding", user_dnn_output)

model.__setattr__("item_input", item_inputs_list)
model.__setattr__("item_embedding",
                  get_item_embedding(pooling_item_embedding_weight, item_features[item_feature_name]))


# Model Compile and Training

In [147]:
model.compile(optimizer="adam", loss=sampledsoftmaxloss)

In [148]:
history = model.fit(train_model_input, train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )

Train on 44840 samples
Epoch 1/20
44840/44840 [==============================] - 4s 96us/sample - loss: 4.8383
Epoch 2/20
44840/44840 [==============================] - 4s 81us/sample - loss: 4.7749
Epoch 3/20
44840/44840 [==============================] - 4s 80us/sample - loss: 4.6907
Epoch 4/20
44840/44840 [==============================] - 4s 80us/sample - loss: 4.6107
Epoch 5/20
44840/44840 [==============================] - 5s 103us/sample - loss: 4.5664
Epoch 6/20
44840/44840 [==============================] - 4s 97us/sample - loss: 4.5114
Epoch 7/20
44840/44840 [==============================] - 4s 81us/sample - loss: 4.4921
Epoch 8/20
44840/44840 [==============================] - 4s 80us/sample - loss: 4.4613
Epoch 9/20
44840/44840 [==============================] - 4s 80us/sample - loss: 4.4478
Epoch 10/20
44840/44840 [==============================] - 4s 80us/sample - loss: 4.4294
Epoch 11/20
44840/44840 [==============================] - 4s 80us/sample - loss: 4.4248
Epoch 

In [159]:
res = model.predict(test_model_input)

In [160]:
res.shape

(586, 1)

# Predict: Generate Item and User Embeddings

In [151]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"sku_number": item_profile['sku_number'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(586, 32)
(8036, 32)


In [161]:
prob_matrix = user_embs.dot(item_embs.T)
prob_matrix.shape

(586, 8036)

# Recommendation

In [162]:
! pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 14.7 MB/s 


In [164]:
import numpy as np
import faiss
from tqdm import tqdm
# from deepmatch.utils import recall_N

In [165]:
def recall_N(y_true, y_pred, N=50):
    return len(set(y_pred[:N]) & set(y_true)) * 1.0 / len(y_true)

In [169]:
test_true_label = {line[0]:[line[1]] for line in test_set}

import numpy as np
import faiss
from tqdm import tqdm
# from deepmatch.utils import recall_N

index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)
# faiss.normalize_L2(user_embs)
D, I = index.search(np.ascontiguousarray(user_embs), 50)
s = []
hit = 0
pred_label = {}
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
    try:
        pred = [item_profile['sku_number'].values[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=50)
        s.append(recall_score)
        pred_label[uid] = pred
        if test_true_label[uid] in pred:
            hit += 1
    except:
        print(i)
print("")
print("recall", np.mean(s))
print("hit rate", hit / len(test_user_model_input['user_id']))

586it [00:00, 3064.43it/s]


recall 0.051194539249146756
hit rate 0.051194539249146756


In [175]:
len(pred_label[160])

50

# Validation

In [176]:
df_train = pd.DataFrame(train_set)

In [178]:
test_user_model_input['user_id'][0:10]

array([ 95, 117,  31,  64,  55, 114,  79,  64,  64,  64])

In [179]:
user_idx = 0
uid = 95

In [180]:
D[user_idx]

array([0.8632025 , 0.8624981 , 0.86122733, 0.8571665 , 0.85410684,
       0.8459642 , 0.84406877, 0.841776  , 0.84042025, 0.8396632 ,
       0.8394666 , 0.8320652 , 0.8265535 , 0.8243533 , 0.81347454,
       0.80929357, 0.80869955, 0.79558605, 0.79115045, 0.79062504,
       0.7823014 , 0.78130853, 0.77775866, 0.7768352 , 0.76669   ,
       0.76597196, 0.7634426 , 0.7630542 , 0.7612563 , 0.7563493 ,
       0.75624573, 0.7497934 , 0.7443189 , 0.7424351 , 0.74174327,
       0.74086237, 0.74052584, 0.74051046, 0.74021465, 0.7393544 ,
       0.7391091 , 0.73826045, 0.7378772 , 0.73642945, 0.7354745 ,
       0.7346438 , 0.7345009 , 0.73408306, 0.7322324 , 0.7312832 ],
      dtype=float32)

In [181]:
sorted_index = prob_matrix.argsort()[:,-50:]
-np.sort(-prob_matrix[user_idx][sorted_index[user_idx]])

array([0.8632025 , 0.8624981 , 0.86122733, 0.8571665 , 0.85410684,
       0.8459642 , 0.84406877, 0.841776  , 0.84042025, 0.8396632 ,
       0.8394666 , 0.8320652 , 0.8265535 , 0.8243533 , 0.81347454,
       0.80929357, 0.80869955, 0.79558605, 0.79115045, 0.79062504,
       0.7823014 , 0.78130853, 0.77775866, 0.7768352 , 0.76669   ,
       0.76597196, 0.7634426 , 0.7630542 , 0.7612563 , 0.7563493 ,
       0.75624573, 0.7497934 , 0.7443189 , 0.7424351 , 0.74174327,
       0.74086237, 0.74052584, 0.74051046, 0.74021465, 0.7393544 ,
       0.7391091 , 0.73826045, 0.7378772 , 0.73642945, 0.7354745 ,
       0.7346438 , 0.7345009 , 0.73408306, 0.7322324 , 0.7312832 ],
      dtype=float32)

In [184]:
I[user_idx]

array([3166, 1102, 5035,  269,  272, 4305, 3167,  273, 3307,  411, 5196,
        271, 3464, 3025,  268, 6422, 1159, 1620,   76, 6723, 4791, 7713,
       4105, 6590, 2895,  129, 1103,  352, 5036, 7605, 6493, 1158, 5635,
       1631,   94,  221,  367, 1160,  686,  188, 2897, 6510, 6218, 1447,
        509,  148, 1342, 1088, 1647,  267])

In [185]:
(sorted_index[user_idx])[::-1]

array([3166, 1102, 5035,  269,  272, 4305, 3167,  273, 3307,  411, 5196,
        271, 3464, 3025,  268, 6422, 1159, 1620,   76, 6723, 4791, 7713,
       4105, 6590, 2895,  129, 1103,  352, 5036, 7605, 6493, 1158, 5635,
       1631,   94,  221,  367, 1160,  686,  188, 2897, 6510, 6218, 1447,
        509,  148, 1342, 1088, 1647,  267])

In [186]:
np.array([item_profile['sku_number'].values[x] for x in I[user_idx]])

array([5158, 5119, 5120, 5133, 5132, 5168, 5143, 5973, 6105, 4934, 4512,
       5118, 5121, 3658, 5131, 6680, 5142, 5925, 4930, 6686, 4845, 6681,
       1056, 7595, 6088, 3520, 5145, 7661, 5148, 6132, 4921, 5974, 7438,
       3517, 6091, 3652, 1912, 3653, 1153, 3518, 3665, 6278, 5497, 6147,
       4842, 5130, 6156, 3833, 5157, 3871])

In [188]:
len(set(df_train[df_train[0]==uid][1].unique()) & set([item_profile['sku_number'].values[x] for x in I[user_idx]]))

2

In [189]:
len(set(df_train[df_train[0]==uid][1].unique()) & set(sorted_index[user_idx]))

1

In [190]:
item_profile['sku_number']

8530     3075
8547     2387
8548     2351
8549      985
8550     4559
         ... 
10985    3921
10998    7852
10992    7557
10960    4007
10961    5693
Name: sku_number, Length: 8036, dtype: int64

In [196]:
all_item_model_input

{'sku_number': array([3075, 2387, 2351, ..., 7557, 4007, 5693])}